In [1]:
%pip install dill
%pip install nltk
%pip install inflect
%pip install pydantic

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# import sys
# import openai

# sys.path.insert(0, "../")
# sys.path.insert(0, "../..")
# from GenAgentsBoardroom.text_adventure_games.parsing import GptParser3
# from GenAgentsBoardroom.test.game_setup import build_boardroom

# game = build_boardroom(
#     experiment_name="boardroom-test",
#     experiment_id=1,
#     num_characters=1,
#     max_ticks=4,
#     personas_path="game_personas",
#     # random_placement=True,
# )
# game.give_hints = True
# parser = GptParser3(game, verbose=False)
# game.set_parser(parser)
# parser.refresh_command_list()

# try:
#     game.game_loop()
# except openai.APIConnectionError:
#     print("Could not connect to OpenAI API")
# finally:
#     game.save_simulation_data()

In [3]:
import sys
import openai

sys.path.insert(0, "../")
sys.path.insert(0, "../..")
from GenAgentsBoardroom.text_adventure_games.parsing import GptParser3
from GenAgentsBoardroom.test.game_setup import build_classic

game = build_classic(
    experiment_name="classic-survivor",
    experiment_id=6,
    num_characters=3,
    max_ticks=3,
    personas_path="game_personas",
    random_placement=False,
)

game.give_hints = True
parser = GptParser3(game, verbose=False)
game.set_parser(parser)
parser.refresh_command_list()

try:
    game.game_loop()
    
except openai.APIConnectionError:
    print("Could not connect to OpenAI API")
finally:
    game.save_simulation_data()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kylesullivan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kylesullivan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



GET OR CREATE BASE FACTS PLAYER DESCRIPTION: An quirky contestant that is must see TV on a reality show. True
RESPONSE: Name='Benny Sparks' Age=28 Likes=['extreme sports', 'juggling', 'making people laugh'] Dislikes=['boring routines', 'silence', 'close-mindedness'] Occupation='Reality TV Contestant' Home_City='Asheville, North Carolina'
Generated facts: {'Name': 'Benny Sparks', 'Age': 28, 'Likes': ['extreme sports', 'juggling', 'making people laugh'], 'Dislikes': ['boring routines', 'silence', 'close-mindedness'], 'Occupation': 'Reality TV Contestant', 'Home_City': 'Asheville, North Carolina'}
Character Benny Sparks starts at Camp and belongs to Group C

GET OR CREATE BASE FACTS PLAYER DESCRIPTION: An quirky contestant that is must see TV on a reality show. True
RESPONSE: Name='Lila Quinn' Age=27 Likes=['dancing', 'talking to plants', 'impressionist painting', 'quirky hats'] Dislikes=['serious people', 'routine', 'judgment', 'boring food'] Occupation='Reality Show Contestant' Home_Ci

In [17]:
import json
from GenAgentsBoardroom.text_adventure_games.parsing import Parser

# Read the JSONL file and print the 'message' field
with open('/Users/kylesullivan/Documents/Professional/LLM Agents/GenAgentsBoardroom/GenAgentsBoardroom/logs/classic-survivor-6/gpt_call_classic-survivor-6.jsonl', 'r') as file:
    for i, line in enumerate(file):
        if i >= 50:  # Only read the first 50 lines
            break
        data = json.loads(line)
        message = data.get('messages', 'No message found')
        print(Parser.wrap_text(message))
        print(f"{'-'*35} Response {'-'*35}\n")
        print(Parser.wrap_text(data.get('response', 'No response found')))
        print('\n' + '=' * 80 + '\n')
        
# Resort shown goals to have the highest priority goals at the top (less likely to be trimmed)

*************************System*************************:

WORLD INFO:
You are competing in a game show where you can be voted out by your fellow
contestants at the end of a round. There are currently 3 more contestants,
including you. The others are: Lila Quinn who is at Camp, Lacey Daydream who is
at Camp. You may only talk to characters in your same location. If they are not
here, then you need to move first to where their location. When 2 contestants
remain, a jury of the contestants you have voted off will vote for a winner!
This means there are 1 rounds until the finals. You have 4 more turns this round
before you will vote. There could be an idol hiding somewhere in the game that
grants you immunity from a vote when you have it in your possession. Explore to
find it!!

You are Lacey Daydream, a 27-year-old Reality Show Contestant. You are
passionate about spontaneous dance parties, collecting vintage hats, telling dad
jokes. You have aversions to sitting still, serious conversat